In [92]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json
#from selenium.webdriver.support.expected_conditions import element_selection_state_to_be
#import matplotlib.pyplot as plt
import base64
#import matplotlib.pyplot as plt
import io
import os
from math import floor

## web crawling packages
import time
from datetime import date
from datetime import datetime
import datetime
import requests
from lxml import html
import csv
import urllib.request
from html_table_parser.parser import HTMLTableParser
from pprint import pprint

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import chromedriver_autoinstaller
import selenium

## scrapping game id's from espn

In [ ]:
//*[@id="401326436"]
//*[@id="401326436"]
/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section

In [ ]:
/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div

In [187]:
driver_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(driver_path)
driver.get('https://www.espn.com/nfl/scoreboard/_/week/8/year/2021/seasontype/2')

In [53]:
test = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section')

test_list = []
for p in range(len(test)):
    test_list.append(test[p].get_attribute("id"))

test_list
print(len(test_list))

3


In [43]:
first = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section')

first_list = []
for p in range(len(first)):
    first_list.append(first[p].get_attribute("id"))

first_list

['401326436']

In [44]:
sunday = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[2]/div/section')

sunday_list = []
for p in range(len(sunday)):
    sunday_list.append(sunday[p].get_attribute("id"))

sunday_list

['401326437',
 '401326438',
 '401326439',
 '401326440',
 '401326441',
 '401326443',
 '401326444',
 '401326442',
 '401326445',
 '401326446',
 '401326447',
 '401326448',
 '401326449']

In [45]:
monday = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[3]/div/section')

monday_list = []
for p in range(len(monday)):
    monday_list.append(monday[p].get_attribute("id"))

monday_list

['401326450']

## starting code for the loop here

In [176]:
url_list = []

year_list = ['2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
#year_list = ['2019', '2020', '2021']

for i in year_list:
    year = str(i)

    for i in range(1,19):
        week = str(i)
        url_list.append([year, week])

url_df = pd.DataFrame(url_list, columns = ['year', 'week'])
url_df['url'] = "https://www.espn.com/nfl/scoreboard/_/week/" + url_df['week'] + "/year/" + url_df['year'] + "/seasontype/2"
print(url_df.shape)
url_df.head()

(360, 3)


,year,week,url
0,2002,1,https://www.espn.com/nfl/scoreboard/_/week/1/y...
1,2002,2,https://www.espn.com/nfl/scoreboard/_/week/2/y...
2,2002,3,https://www.espn.com/nfl/scoreboard/_/week/3/y...
3,2002,4,https://www.espn.com/nfl/scoreboard/_/week/4/y...
4,2002,5,https://www.espn.com/nfl/scoreboard/_/week/5/y...


In [177]:
game_id_df = pd.DataFrame()

for i in url_df['url']:
    driver.get(i)

    ## temp dataframes as a holding place before appending all together at the end of the loop
    first_df_temp = pd.DataFrame()
    second_df_temp = pd.DataFrame()
    third_df_temp = pd.DataFrame()
    fourth_df_temp = pd.DataFrame()
    
    try:
        first_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[1]/div/section')

        first_list = []
        for p in range(len(first_day)):
            first_list.append(first_day[p].get_attribute("id"))
        first_df = pd.DataFrame(first_list, columns = ['id'])
        first_df_temp = first_df_temp.append(first_df)

    except Exception as e:
        print("Error Couldn't Find First element:", e)

    try:
        second_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[2]/div/section')

        second_list = []
        for p in range(len(second_day)):
            second_list.append(second_day[p].get_attribute("id"))
        second_df = pd.DataFrame(second_list, columns = ['id'])
        second_df_temp = second_df_temp.append(second_df)

    except Exception as e:
        print("Error Couldn't Find Second element:", e)

    try:
        third_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[3]/div/section')

        third_list = []
        for p in range(len(third_day)):
            third_list.append(third_day[p].get_attribute("id"))
        third_df = pd.DataFrame(third_list, columns = ['id'])
        third_df_temp = third_df_temp.append(third_df)

    except Exception as e:
        print("Error Couldn't Find Third element:", e)

    try:
        fourth_day = driver.find_elements_by_xpath('/html/body/div[1]/div/div/div/div[4]/div[3]/div/div[1]/div[1]/div/section[4]/div/section')

        fourth_list = []
        for p in range(len(fourth_day)):
            fourth_list.append(fourth_day[p].get_attribute("id"))
        fourth_df = pd.DataFrame(fourth_list, columns = ['id'])
        fourth_df_temp = fourth_df_temp.append(fourth_df)

    except Exception as e:
        print("Error Couldn't Find Fourth element:", e)

    game_id_df_temp = pd.DataFrame(first_df_temp.append([second_df_temp, third_df_temp, fourth_df_temp])).reset_index()
    game_id_df_temp['url'] = i
    game_id_df_temp = game_id_df_temp.drop(columns = ['index'])

    game_id_df = game_id_df.append(game_id_df_temp).reset_index()
    game_id_df = game_id_df.drop(columns = ['index'])

In [178]:
print(game_id_df.shape)
game_id_df.tail()

(5137, 2)


,id,url
5132,401326590,https://www.espn.com/nfl/scoreboard/_/week/18/...
5133,401326598,https://www.espn.com/nfl/scoreboard/_/week/18/...
5134,401326600,https://www.espn.com/nfl/scoreboard/_/week/18/...
5135,401326599,https://www.espn.com/nfl/scoreboard/_/week/18/...
5136,401326597,https://www.espn.com/nfl/scoreboard/_/week/18/...


In [180]:
game_id_df.to_csv('data/game_id_all.csv')

## Scrape Game Odds

In [124]:
## hitting this api will show the current weeks games and the betting data behind each game

driver.get('https://www.espn.com/nfl/scoreboard')

url = "https://site.web.api.espn.com/apis/v2/scoreboard/header?sport=football&league=nfl&region=us&lang=en&contentorigin=espn&buyWindow=1m&showAirings=buy%2Clive%2Creplay&showZipLookup=true&tz=America/New_York"

payload={}
headers = {
  'Accept': 'application/json',
  'Cookie': 'SWID=3750444F-BA06-4929-C99C-62333D7EE8A0'
}

response = requests.request("GET", url, headers=headers, data=payload)

response_text = json.loads(response.text)

game_df = json_normalize(response_text['sports'], ['leagues', 'events'])

## include this data after predictions to find valuable bets that show some different towhat vegas thinks

In [127]:
game_df.head()

,gamecastAvailable,playByPlayAvailable,commentaryAvailable,recent,id,competitionId,uid,date,timeValid,name,...,odds.homeTeamOdds.underdog,odds.homeTeamOdds.moneyLine,odds.homeTeamOdds.spreadOdds,odds.homeTeamOdds.team.id,odds.homeTeamOdds.team.abbreviation,odds.links,odds.moneylineWinner,odds.spreadWinner,odds.home.moneyLine,odds.away.moneyLine
0,False,False,False,False,401326466,401326466,s:20~l:28~e:401326466~c:401326466,2021-11-14T18:00:00Z,True,Atlanta Falcons at Dallas Cowboys,...,False,-420,-110.0,6,DAL,[],False,False,-420,320
1,False,False,False,False,401326471,401326471,s:20~l:28~e:401326471~c:401326471,2021-11-14T18:00:00Z,True,New Orleans Saints at Tennessee Titans,...,False,-160,100.0,10,TEN,[],False,False,-160,140
2,False,False,False,False,401326467,401326467,s:20~l:28~e:401326467~c:401326467,2021-11-14T18:00:00Z,True,Jacksonville Jaguars at Indianapolis Colts,...,False,-475,-110.0,11,IND,[],False,False,-475,360
3,False,False,False,False,401326468,401326468,s:20~l:28~e:401326468~c:401326468,2021-11-14T18:00:00Z,True,Cleveland Browns at New England Patriots,...,False,-140,-110.0,17,NE,[],False,False,-140,120
4,False,False,False,False,401326469,401326469,s:20~l:28~e:401326469~c:401326469,2021-11-14T18:00:00Z,True,Buffalo Bills at New York Jets,...,True,550,-110.0,20,NYJ,[],False,False,550,-800


## scraping individual game data -- we just need to find a way we get scrap all the game id's and then pass them to the functions below

In [3]:
# Opens a website and read its
# binary contents (HTTP Response Body)
def url_get_contents(url):

    # Opens a website and read its
    # binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [171]:
# defining the html contents of a URL.
xhtml = url_get_contents('https://www.espn.com/nfl/matchup?gameId=401326425').decode('utf-8')

# Defining the HTMLTableParser object
p = HTMLTableParser()

# feeding the html contents in the
# HTMLTableParser object
p.feed(xhtml)

# Now finally obtaining the data of
# the table required
#pprint(p.tables[1])

df_testing = pd.DataFrame(p.tables[1])
column_list = list(df_testing[0].values)

away = pd.DataFrame(df_testing[1]).T
away.columns = column_list
away = away.reset_index()
away = away.drop(columns = ['index', 'Matchup'])
#away.drop(away.columns[[13]], axis=1, inplace=True)
away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

home = pd.DataFrame(df_testing[2]).T
home.columns = column_list
home = home.reset_index()
home = home.drop(columns = ['index', 'Matchup'])
#home.drop(home.columns[[13]], axis=1, inplace=True)
home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

game_stat_temp = pd.concat([away, home], axis=1)
game_stat_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 50 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   first_downs_away            1 non-null      object
 1   first_downs_passing_away    1 non-null      object
 2   first_down_rushing_away     1 non-null      object
 3   first_down_by_penalty_away  1 non-null      object
 4   third_down_away             1 non-null      object
 5   fourth_down_away            1 non-null      object
 6   total_plays_away            1 non-null      object
 7   total_yards_away            1 non-null      object
 8   total_drives_away           1 non-null      object
 9   yards_per_play_away         1 non-null      object
 10  passing_away                1 non-null      object
 11  comp_att_away               1 non-null      object
 12  yards_per_pass_away         1 non-null      object
 13  int_thrown_away             1 non-null      object
 14

In [26]:
print('Level Two gets you each row...', p.tables[1][1])
print('row header...', p.tables[1][1][0])
print('away data point...', p.tables[1][1][1])
print('home data point...', p.tables[1][1][2])

Level Two gets you each row... ['1st Downs', '25', '19']
row header... 1st Downs
away data point... 25
home data point... 19


### need to just write a loop to go through this table and extract all points

In [188]:
game_stats_big_df = pd.DataFrame()

for i in game_id_df['id']:
    
    try:
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        # defining the html contents of a URL.
        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        # Now finally obtaining the data of
        # the table required
        df_testing = pd.DataFrame(p.tables[1])
        column_list = list(df_testing[0].values)

        away = pd.DataFrame(df_testing[1]).T
        away.columns = column_list
        away = away.reset_index()
        away = away.drop(columns = ['index', 'Matchup'])
        #away.drop(away.columns[[13]], axis=1, inplace=True)
        away = away.rename(columns = {'1st Downs': 'first_downs_away',
                            'Passing 1st downs': 'first_downs_passing_away',
                            'Rushing 1st downs': 'first_downs_rushing_away', 
                            '1st downs from penalties': 'first_down_by_penalty_away', 
                            '3rd down efficiency': 'third_down_away', 
                            '4th down efficiency': 'fourth_down_away', 
                            'Total Plays': 'total_plays_away', 
                            'Total Yards': 'total_yards_away', 
                            'Total Drives': 'total_drives_away', 
                            'Yards per Play': 'yards_per_play_away', 
                            'Passing': 'passing_away', 
                            'Comp-Att': 'comp_att_away', 
                            'Yards per pass': 'yards_per_pass_away',
                            'Fumbles Lost': 'fumbles_lost_away',  
                            'Sacks-Yards Lost': 'sack_yards_lost_away', 
                            'Rushing': 'rushing_away', 
                            'Rushing Attempts': 'rushing_att_away', 
                            'Yards per rush': 'yards_per_rush_away', 
                            'Red Zone (Made-Att)': 'red_zone_att_away', 
                            'Penalties': 'penalty_away', 
                            'Turnovers': 'turnovers_away', 
                            'Interceptions thrown': 'int_thrown_away', 
                            'Rushing 1st downs': 'first_down_rushing_away', 
                            'Defensive / Special Teams TDs': 'defensive_td_away', 
                            'Possession': 'possession_away'})

        home = pd.DataFrame(df_testing[2]).T
        home.columns = column_list
        home = home.reset_index()
        home = home.drop(columns = ['index', 'Matchup'])
        #home.drop(home.columns[[13]], axis=1, inplace=True)
        home = home.rename(columns = {'1st Downs': 'first_downs_home',
                            'Passing 1st downs': 'first_downs_passing_home',
                            'Rushing 1st downs': 'first_downs_rushing_home', 
                            '1st downs from penalties': 'first_down_by_penalty_home', 
                            '3rd down efficiency': 'third_down_home', 
                            '4th down efficiency': 'fourth_down_home', 
                            'Total Plays': 'total_plays_home', 
                            'Total Yards': 'total_yards_home', 
                            'Total Drives': 'total_drives_home', 
                            'Yards per Play': 'yards_per_play_home', 
                            'Passing': 'passing_home', 
                            'Comp-Att': 'comp_att_home', 
                            'Yards per pass': 'yards_per_pass_home',
                            'Fumbles Lost': 'fumbles_lost_home',  
                            'Sacks-Yards Lost': 'sack_yards_lost_home', 
                            'Rushing': 'rushing_home', 
                            'Rushing Attempts': 'rushing_att_home', 
                            'Yards per rush': 'yards_per_rush_home', 
                            'Red Zone (Made-Att)': 'red_zone_att_home', 
                            'Penalties': 'penalty_home', 
                            'Turnovers': 'turnovers_home', 
                            'Interceptions thrown': 'int_thrown_home', 
                            'Rushing 1st downs': 'first_down_rushing_home', 
                            'Defensive / Special Teams TDs': 'defensive_td_home', 
                            'Possession': 'possession_home'})

        game_stat_temp = pd.concat([away, home], axis=1)
        game_stat_temp['id'] = i
        game_stats_big_df = game_stats_big_df.append(game_stat_temp)

    except Exception as e:
        print("Error:", e)
    

Error: "['Matchup'] not found in axis"
Error: HTTP Error 503: Service Unavailable
Error: HTTP Error 503: Service Unavailable
Error: HTTP Error 503: Service Unavailable
Error: HTTP Error 503: Service Unavailable
Error: HTTP Error 503: Service Unavailable
Error: "['Matchup'] not found in axis"
Error: HTTP Error 503: Service Unavailable
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: HTTP Error 503: Service Unavailable
Error: list index out of range
Error: "['Matchup'] not found in axis"
Error: HTTP Error 503: Service Unavailable
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
Error: "['Matchup'] not found in axis"
E

In [190]:
game_stats_big_df.to_csv('data/game_stats_all.csv')


In [189]:
game_stats_big_df.head()

,first_downs_away,first_downs_passing_away,first_down_rushing_away,first_down_by_penalty_away,third_down_away,fourth_down_away,total_plays_away,total_yards_away,total_drives_away,yards_per_play_away,...,rushing_att_home,yards_per_rush_home,red_zone_att_home,penalty_home,turnovers_home,Fumbles lost,int_thrown_home,defensive_td_home,possession_home,id
0,13,7,5,1,4-12,0-0,51,279,13,5.5,...,22,2.0,0-6,10-80,3,0,3,0,32:28,220905019
0,18,11,4,3,2-8,0-0,47,266,17,5.7,...,32,4.4,0-8,10-82,3,1,2,0,39:08,220908002
0,19,9,9,1,5-13,0-0,62,368,17,5.9,...,26,3.1,0-6,4-33,2,1,1,0,28:29,220908003
0,27,9,16,2,6-10,0-0,65,401,11,6.2,...,13,2.8,0-5,9-57,1,0,1,0,22:12,220908004
0,24,12,10,2,5-11,0-0,59,470,20,8.0,...,20,3.0,0-10,4-38,1,1,0,0,29:44,220908005


In [241]:
game_score_big_df = pd.DataFrame()

for i in game_id_df['id']:
    
    try:
        # defining the html contents of a URL.
        web = "https://www.espn.com/nfl/matchup?gameId=" + str(i)

        xhtml = url_get_contents(web).decode('utf-8')

        # Defining the HTMLTableParser object
        p = HTMLTableParser()

        # feeding the html contents in the
        # HTMLTableParser object
        p.feed(xhtml)

        df_score = pd.DataFrame(p.tables[0])
        df_score = pd.DataFrame(df_score.iloc[1: , :])
        team_column = df_score[0].to_frame()
        team_df = pd.DataFrame(team_column, columns = ['team'])
        score_column = df_score.iloc[: , -1].to_frame()
        score_df = pd.DataFrame(score_column, columns = ['score'])
        df_score_new = pd.concat([team_df, score_df], axis=1)

        away_score = pd.DataFrame(df_score_new.iloc[0]).T
        away_score = away_score.reset_index()
        away_score = away_score[['team', 'score']]
        away_score = away_score.rename(columns = {'team': 'away_team', 'score': 'away_score'})

        home_score = pd.DataFrame(df_score_new.iloc[1]).T
        home_score = home_score.reset_index()
        home_score = home_score[['team', 'score']]
        home_score = home_score.rename(columns = {'team': 'home_team', 'score': 'home_score'})

        game_score_temp = pd.concat([away_score, home_score], axis=1)
        game_score_temp['id'] = i
        game_score_big_df = game_score_big_df.append(game_score_temp)

    except Exception as e:
        print('Error:', e, 'game_id=', i)

In [226]:
game_score_big_df

,away_team,away_score,home_team,home_score,id
0,SF,16,NYG,13,220905019
